In [8]:
import numpy as np
import random
import pydot
from IPython.display import display, SVG, clear_output

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (Parser, StartMeshcat, DiagramBuilder,
                        MeshcatVisualizerCpp, JacobianWrtVariable,
                        MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.geometry import (Box, Cylinder)
from pydrake.multibody.tree import (PrismaticJoint, UnitInertia, SpatialInertia, RevoluteJoint, FixedOffsetFrame, WeldJoint)
from manipulation.meshcat_cpp_utils import (MeshcatJointSliders)
from manipulation.scenarios import AddMultibodyTriad
# from manipulation.meshcat_cpp_utils import (StartMeshcat, MeshcatJointSlidersThatPublish)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [9]:
# Build robot model in Drake
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

# parameters
LL = 0.4  # lenght of cylinder
L0 = 1
L1 = 1
L2 = 1
L3 = 1

RGBA_Color0 = [0.5, 0.5, 0.5, 0.4]
RGBA_Color1 = [0, 0.5, 0.5, 0.4]
RGBA_Color2 = [0.9, 0, 0, 0.4]
RGBA_Color3 = [0.9, 0.5, 0, 0.4]

my_model_instance = plant.AddModelInstance("my_robot")

# SpatialInertia(mass, reference point (wrt CoM), UnitInertia()): 6x6 matrix
# This does not matter for kinematics
default_inertia = SpatialInertia(1, [0, 0, L0/2], UnitInertia(1, 1, 1))

link_0 = plant.AddRigidBody("link_0", my_model_instance, default_inertia)

link_1 = plant.AddRigidBody("link_1", my_model_instance, default_inertia)

link_2 = plant.AddRigidBody("link_2", my_model_instance, default_inertia)

link_3 = plant.AddRigidBody("link_3", my_model_instance, default_inertia)

# plant.RegisterVisualGeometry(body,
c = 0.1
c0 = 0.03
plant.RegisterVisualGeometry(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Cylinder(c0, L0),
    "link_0",
    RGBA_Color0
)
plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Cylinder(c, LL),
    "link_1",
    RGBA_Color1)
plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [L1/2, 0, 0]),
    Cylinder(c0, L1),
    "link_1_1",
    RGBA_Color1)
plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Cylinder(c, LL),
    "link_2",
    RGBA_Color2)
plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [L2/2, 0, 0]),
    Cylinder(c0, L2),
    "link_2_2",
    RGBA_Color2)
plant.RegisterVisualGeometry(
    link_3,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Cylinder(c, LL),
    "link_3",
    RGBA_Color3)
plant.RegisterVisualGeometry(
    link_3,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Cylinder(c0, L3),
    "link_3_3",
    RGBA_Color3)

Joint1_Frame = plant.AddFrame(FixedOffsetFrame(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0])))

Joint2_Frame = plant.AddFrame(FixedOffsetFrame(
    link_1,
    RigidTransform(RollPitchYaw(np.pi/2, np.pi/2, 0), [L1, 0, 0])))

Joint3_Frame = plant.AddFrame(FixedOffsetFrame(
    link_2,
    RigidTransform(RollPitchYaw(-np.pi/2, 0, 0), [L2, 0, 0])))

Joint1 = plant.AddJoint(RevoluteJoint(
    name="Joint1", frame_on_parent=Joint1_Frame,
    frame_on_child=link_1.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint2 = plant.AddJoint(RevoluteJoint(
    name="Joint2", frame_on_parent=Joint2_Frame,
    frame_on_child=link_2.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint3 = plant.AddJoint(RevoluteJoint(
    name="Joint3", frame_on_parent=Joint3_Frame,
    frame_on_child=link_3.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))

plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=link_0.body_frame(),
    X_PC=RigidTransform.Identity())


# add frames of interest
G = plant.AddFrame(FixedOffsetFrame('EndEffector',
                                    link_3,
                                    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0])))
                                    # Draw RGB frames for visualization
                                    
for body_name in ["link_0", "link_1","link_2", "EndEffector"]:
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph, 0.20, 0.008)


In [10]:
# Finalize and visualize

plant.Finalize()
renderer_name = "renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))
meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.Publish(diagram_context)

In [11]:
# Get results frome drake
def SE3fromDrake(theta1,theta2,theta3):
    plant.SetPositions(plant.GetMyContextFromRoot(diagram_context), my_model_instance, [theta1, theta2,theta3]) # theta1, theta2
    diagram.Publish(diagram_context)
    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
    X_WG=G.CalcPoseInWorld(plant_context)
    return X_WG

def SptailJacobianfromeDrake(theta1,theta2,theta3):
    T=SE3fromDrake(theta1,theta2,theta3)    #end-effector SE3
    W = plant.world_frame()
    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
    J_G = plant.CalcJacobianSpatialVelocity(plant_context, JacobianWrtVariable.kQDot, G, T.inverse().translation(), W, W)
    return J_G
    
# My forwardkinematics and jacobian
import modern_robotics as mr
def myfwd(theta1,theta2,theta3):
    # step 0: compute M 
    M = mr.RpToTrans(np.array([[0,0,1],[0,1,0],[-1,0,0]]), np.array([L1, 0, -L2]))
    # step 1: compute all the screw axis
    Sbar_1_0=mr.ScrewToAxis(np.array([0,0,0]),np.array([0,0,1]),0)
    Sbar_2_0=mr.ScrewToAxis(np.array([L1,0,0]),np.array([0,-1,0]),0)
    Sbar_3_0=mr.ScrewToAxis(np.array([0,0,-L2]),np.array([1,0,0]),0)
    def myPoE(theta1,theta2,theta3):
        SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
        SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
        SbarMatrix_3_0=mr.VecTose3(Sbar_3_0)
        return mr.MatrixExp6(SbarMatrix_1_0*theta1)@mr.MatrixExp6(SbarMatrix_2_0*theta2)@mr.MatrixExp6(SbarMatrix_3_0*theta3)@M
    Tbs = myPoE(theta1,theta2,theta3)
    return Tbs
def myspatialjacobian(theta1,theta2,theta3):
    Sbar_1_0=mr.ScrewToAxis(np.array([0,0,0]),np.array([0,0,1]),0)
    Sbar_2_0=mr.ScrewToAxis(np.array([L1,0,0]),np.array([0,-1,0]),0)
    Sbar_3_0=mr.ScrewToAxis(np.array([0,0,-L2]),np.array([1,0,0]),0)
    SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
    SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
    S_1=Sbar_1_0
    S_2=mr.Adjoint(mr.MatrixExp6(SbarMatrix_1_0*theta1))@Sbar_2_0
    S_3=mr.Adjoint(mr.MatrixExp6(SbarMatrix_1_0*theta1)@mr.MatrixExp6(SbarMatrix_2_0*theta2))@Sbar_3_0
    J_s=np.array([S_1,S_2,S_3]).transpose()
    return J_s

## (a) & (b)

In [12]:

np.set_printoptions(suppress=True)
for i in range(2):
    theta1 = -np.pi+random.random()*2*np.pi
    theta2 = -np.pi+random.random()*2*np.pi
    theta3 = -np.pi+random.random()*2*np.pi
    print("My forward kinematics at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
    print(myfwd(theta1,theta2,theta3))
    print("Drake forward kinematics at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
    print(SE3fromDrake(theta1,theta2,theta3).GetAsMatrix4())

My forward kinematics at theta = [ 0.44886971 -1.84554614 -1.58002145]:
[[ 0.44192845 -0.86310671 -0.24443006  0.03379139]
 [-0.89704681 -0.42596484 -0.11773262  0.01627602]
 [-0.0025028   0.2712946  -0.9624931   0.27130614]
 [ 0.          0.          0.          1.        ]]
Drake forward kinematics at theta = [ 0.44886971 -1.84554614 -1.58002145]:
[[ 0.44192845 -0.86310671 -0.24443006  0.03379139]
 [-0.89704681 -0.42596484 -0.11773262  0.01627602]
 [-0.0025028   0.2712946  -0.9624931   0.27130614]
 [ 0.          0.          0.          1.        ]]
My forward kinematics at theta = [-0.57590066  2.57103626  1.85178933]:
[[ 0.39761511 -0.58623788 -0.70585223  1.29168552]
 [ 0.88737482  0.05001737  0.4583276  -0.83872388]
 [-0.23338413 -0.80859348  0.54010039  0.8416006 ]
 [ 0.          0.          0.          1.        ]]
Drake forward kinematics at theta = [-0.57590066  2.57103626  1.85178933]:
[[ 0.39761511 -0.58623788 -0.70585223  1.29168552]
 [ 0.88737482  0.05001737  0.4583276  -0

## (c)

In [13]:
for i in range(2):
    theta1 = -np.pi+random.random()*2*np.pi
    theta2 = -np.pi+random.random()*2*np.pi
    theta3 = -np.pi+random.random()*2*np.pi
    print("My Geometric Jacobian at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
    print(myspatialjacobian(theta1,theta2,theta3))
    print("Drake Geometric Jacobian at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
    print(SptailJacobianfromeDrake(theta1,theta2,theta3))

My Geometric Jacobian at theta = [0.89641344 0.70084579 0.265517  ]:
[[ 0.          0.78109244  0.47723885]
 [ 0.         -0.62441541  0.59698664]
 [ 1.          0.          0.64486436]
 [ 0.          0.          1.28479111]
 [ 0.          0.         -1.02707865]
 [ 0.         -1.         -0.        ]]
Drake Geometric Jacobian at theta = [0.89641344 0.70084579 0.265517  ]:
[[ 0.          0.78109244  0.47723885]
 [ 0.         -0.62441541  0.59698664]
 [ 1.          0.          0.64486436]
 [ 0.          0.          1.28479111]
 [ 0.          0.         -1.02707865]
 [ 0.         -1.         -0.        ]]
My Geometric Jacobian at theta = [ 0.63981546 -0.23409188 -1.15499219]:
[[ 0.          0.59704741  0.7803261 ]
 [ 0.         -0.80220595  0.58076318]
 [ 1.          0.         -0.23195973]
 [ 0.          0.          0.45855646]
 [ 0.          0.         -0.61612648]
 [ 0.         -1.          0.        ]]
Drake Geometric Jacobian at theta = [ 0.63981546 -0.23409188 -1.15499219]:
[[ 0.  

## (d)

In [14]:
def axis2skew(w):
    return np.array([[0,-w[2],w[1]],[w[2],0,-w[0]],[-w[1],w[0],0]])

def myanalyticaljacobian(theta1,theta2,theta3):
    T=myfwd(theta1,theta2,theta3)
    pb=np.array([1,2,3,1])
    E=np.zeros((3,6))
    E[0:3,0:3]=-axis2skew((T@pb)[0:3])
    E[0:3,3:6]=np.identity(3)
    J_an= E@myspatialjacobian(theta1,theta2,theta3)
    return J_an
def AnalyticalJacobianfromeDrake(theta1,theta2,theta3):
    # T=SE3fromDrake(theta1,theta2,theta3)    #end-effector SE3
    W = plant.world_frame()
    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
    J_G = plant.CalcJacobianSpatialVelocity(plant_context, JacobianWrtVariable.kQDot, G, [1,2,3], W, W)
    return J_G

print("My Analytical Jacobian at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
print(myanalyticaljacobian(theta1,theta2,theta3))
print("Drake Analytical Jacobian at theta = "+np.array2string(np.array([theta1,theta2,theta3]))+":")
print(AnalyticalJacobianfromeDrake(theta1,theta2,theta3)[3:6,:])

My Analytical Jacobian at theta = [ 0.63981546 -0.23409188 -1.15499219]:
[[-1.80573687  3.08143011 -1.35341036]
 [ 2.60534305  2.29337599  1.77692416]
 [ 0.          2.16813223 -0.1040237 ]]
Drake Analytical Jacobian at theta = [ 0.63981546 -0.23409188 -1.15499219]:
[[-1.80573687  3.08143011 -1.35341036]
 [ 2.60534305  2.29337599  1.77692416]
 [ 0.          2.16813223 -0.1040237 ]]
